<a href="https://colab.research.google.com/github/TomasWW/llm-rag-demo/blob/main/rag_llm_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Install required packages
!pip install -U -q langchain langchain-openai langchain-community qdrant-client pypdf openai

In [30]:
import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# Load your OpenAI API key from Colab secrets
OPENAI_API_KEY = userdata.get('apikey')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
OPENROUTER_API_BASE = "https://openrouter.ai/api/v1"

# Initialize LLM
llm = ChatOpenAI(model="mistralai/mistral-7b-instruct",
                 temperature=0.7,
                 openai_api_base=OPENROUTER_API_BASE
                 )

# Example query using only LLM (without RAG)
response = llm.invoke([
    HumanMessage(content="Who won the Mundial de Clubes FIFA 2025")
])
print("LLM-only response:\n", response.content)



LLM-only response:
  I'm sorry for any confusion, but as of the time of this response, the Mundial de Clubes FIFA 2025 has not taken place yet. The 2020 edition was won by Bayern Munich. I'll be happy to help if you have any questions about past or future editions of the tournament!


In [25]:
#Qdrant setup

from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

# Connect to Qdrant vector database
DB_URL = userdata.get('dburl')
RAG_ENGINE_API_KEY = userdata.get('apikeyrag-engine')

qdrant_client = QdrantClient(
    url=DB_URL,
    api_key=RAG_ENGINE_API_KEY
)

COLLECTION_NAME = "mundial_clubes_2025"

# Delete the collection if it already exists
try:
    qdrant_client.delete_collection(collection_name=COLLECTION_NAME)
    print(f"Collection '{COLLECTION_NAME}' deleted.")
except Exception as e:
    print(f"Collection did not exist: {e}")


Collection 'mundial_clubes_2025' deleted.


In [26]:
# Load and split PDF files
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF files
pdf_files = [
    "Anexo_Final de la Copa Mundial de Clubes de la FIFA 2025.pdf",
    "Copa_Mundial_de_Clubes_de_la_FIFA_2025.pdf"
]

docs = []
for file in pdf_files:
    loader = PyPDFLoader(file)
    docs += loader.load()

print(f"Loaded {len(docs)} pages from the PDF files.")

# Split text into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"Created {len(chunks)} text chunks.")


Loaded 4 pages from the PDF files.
Created 33 text chunks.


In [27]:
from langchain.embeddings import HuggingFaceEmbeddings

# Create embeddings using a HuggingFace model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Insert chunks into Qdrant
qdrant = Qdrant.from_documents(
    chunks,
    embeddings,
    location=DB_URL,
    api_key=RAG_ENGINE_API_KEY,
    collection_name=COLLECTION_NAME
)

print(f"Upload complete. {len(chunks)} embeddings inserted into Qdrant.")


/tmp/ipython-input-2638935739.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Upload complete. 33 embeddings inserted into Qdrant.


In [28]:
# Function to retrieve the most relevant chunks for a user question
def get_context(user_question, k=5):
    """
    Retrieve the k most relevant text chunks from Qdrant
    and concatenate them as context for the LLM.
    """
    docs = qdrant.similarity_search(user_question, k=k)
    context = "\n".join([doc.page_content for doc in docs])
    return context


In [32]:
# User question
user_input = "Quien fue el campeon del Mundial de Clubes FIFA 2025"

# Retrieve context from vector DB
context = get_context(user_input)
print("Retrieved context:\n", context, "\n\n\n")

# Ask LLM using retrieved context
response = llm.invoke([
    HumanMessage(content=f"Using the following context:\n{context}\n\nAnswer the question: {user_input}")
])
print("RAG-enhanced response:\n", response.content)

Retrieved context:
 2029
Sitio oficial (https://www.fifa.com/it/tournaments/mens/club-world-cup/usa-2025)
Copa Mundial de Clubes de la FIFA 2025
La Copa Mundial de Clubes de la FIFA 2025 fue la vigésima primera edición del torneointernacional de clubes de fútbol de la FIFA y la primera que es disputada bajo el formato que reúnea 32 equipos de las 6 confederaciones continentales.2
El 16 de diciembre de 2022 el presidente de FIFA Gianni Infantino confirmó la realización delMundial de Clubes con 32 equipos cada cuatro años a partir del verano boreal de 2025, «en una sedeconcreta», sin confirmar a China, en el espacio de la antigua Copa FIFA Confederaciones.17 
Finalmente, el 23 de junio de 2023 el Consejo de la FIFA designó por unanimidad a Estados Unidoscomo anfitrión del nuevo Mundial de Clubes.18
El campeón fue el Chelsea de Inglaterra, tras derrotar al conjunto francés Paris Saint-Germain por3:0 en la final.8 
Estaba proyectado para jugarse desde el 17 de junio al 4 de julio de 2021, 